In [1]:
import findspark
findspark.init()


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as func
from pyspark.sql.functions import * 

In [3]:
spark = SparkSession.builder.getOrCreate()

In [4]:
bigmac_df = spark.read.csv("C:\\Users\\Keyrus\\Documents\\GitHub\\Python-eng-projects\\Projetos_Python\\Analises_tratamento_bases\\datafolder\\mcdonald_data\\BigmacPrice.csv",header=True)

In [5]:
bigmac_df.show(5)

+----------+-------------+---------+-----------+-----------+------------------+
|      date|currency_code|     name|local_price|  dollar_ex|      dollar_price|
+----------+-------------+---------+-----------+-----------+------------------+
|2000-04-01|          ARS|Argentina|        2.5|        1.0|               2.5|
|2000-04-01|          AUD|Australia|       2.59|       1.68|1.5416666666666698|
|2000-04-01|          BRL|   Brazil|       2.95|       1.79|  1.64804469273743|
|2000-04-01|          GBP|  Britain|        1.9|0.632911392|3.0020000019212802|
|2000-04-01|          CAD|   Canada|       2.85|       1.47|  1.93877551020408|
+----------+-------------+---------+-----------+-----------+------------------+
only showing top 5 rows



In [6]:
bigmac_df.printSchema()

root
 |-- date: string (nullable = true)
 |-- currency_code: string (nullable = true)
 |-- name: string (nullable = true)
 |-- local_price: string (nullable = true)
 |-- dollar_ex: string (nullable = true)
 |-- dollar_price: string (nullable = true)



In [7]:
bigmac_df = bigmac_df.withColumn("local_price",func.col("local_price").cast("Float"))
bigmac_df = bigmac_df.withColumn("dollar_ex",func.col("dollar_ex").cast("Float"))
bigmac_df = bigmac_df.withColumn("dollar_price",func.col("dollar_price").cast("Float"))

In [8]:
bigmac_df.printSchema()

root
 |-- date: string (nullable = true)
 |-- currency_code: string (nullable = true)
 |-- name: string (nullable = true)
 |-- local_price: float (nullable = true)
 |-- dollar_ex: float (nullable = true)
 |-- dollar_price: float (nullable = true)



In [25]:
bigmac_df.show(10)

+----------+-------------+--------------+-----------+---------+------------+
|      date|currency_code|          name|local_price|dollar_ex|dollar_price|
+----------+-------------+--------------+-----------+---------+------------+
|2000-04-01|          ARS|     Argentina|        2.5|      1.0|         2.5|
|2000-04-01|          AUD|     Australia|       2.59|     1.68|   1.5416666|
|2000-04-01|          BRL|        Brazil|       2.95|     1.79|   1.6480447|
|2000-04-01|          GBP|       Britain|        1.9|0.6329114|       3.002|
|2000-04-01|          CAD|        Canada|       2.85|     1.47|   1.9387755|
|2000-04-01|          CLP|         Chile|     1260.0|    514.0|    2.451362|
|2000-04-01|          CNY|         China|        9.9|     8.28|   1.1956521|
|2000-04-01|          CZK|Czech Republic|      54.37|     39.1|   1.3905371|
|2000-04-01|          DKK|       Denmark|      24.75|     8.04|   3.0783582|
|2000-04-01|          EUR|     Euro area|       2.56|1.0752689|      2.3808|

In [23]:
bigmac_df_selected = bigmac_df.filter((bigmac_df.dollar_ex >15) & (bigmac_df.local_price>50)).groupBy("currency_code","name").agg(sum("dollar_price"))


In [26]:
bigmac_df_selected.show(10)

+-------------+--------------+------------------+
|currency_code|          name| sum(dollar_price)|
+-------------+--------------+------------------+
|          CLP|         Chile|106.13718390464783|
|          ARS|     Argentina|21.979254961013794|
|          TRY|        Turkey| 7.943038463592529|
|          UYU|       Uruguay|100.64353847503662|
|          JPY|         Japan| 98.91559910774231|
|          TWD|        Taiwan|  75.0512182712555|
|          LKR|     Sri Lanka|   70.685586810112|
|          VND|       Vietnam| 33.17307448387146|
|          CZK|Czech Republic| 93.32883942127228|
|          PHP|   Philippines| 73.78859853744507|
+-------------+--------------+------------------+
only showing top 10 rows



In [27]:
bigmac_df_selected.write.option("header",True).csv("C:\\Users\\Keyrus\\Documents\GitHub\\Python-eng-projects\\Projetos_Python\\Analises_tratamento_bases\\Spark\\bigmac_price_filtered.csv")

In [32]:
bigmac_df_selected.repartition(1).write.csv("big_mac_price")